In [0]:
from IPython.core.display import HTML
css_file = '../../styles/styles.css'
HTML(open(css_file, "r").read())

# Absolute Photometry

<section class="objectives panel panel-warning">
<div class="panel-heading">
<h2><span class="fa fa-certificate"></span>&nbsp;Learning Objectives</h2>
</div>
</section>

> * How to use the ``photutils`` library to detect sources on an image, and measure instrumental magnitudes for many stars
> * Use the ```astroquery``` and ```astropy``` librarys to cross match catalogs on sky position
> * How to compute zeropoints and uncertainties

In the [lecture](http://slittlefair.staff.shef.ac.uk/teaching/phy241/lectures/L07/index.html) we learnt the theory behind precise calibration of photometry, enabling us to put photometric observations onto a standard scale with precisions of better than 1%. In this practical we are going to learn how to produce calibrated photometry using a **simplified method**. This method does not yield the same accuracy as an analysis using observations of primary or secondary standard stars, combined with the use of colour terms. On the other hand, it is much simpler to apply, and even works to produce calibrated photometry through thin cloud. If you are happy to accept accuracies on the order of a few percent, I recommend using the method described below whenever you need to produce calibrated photometry. 

In this practical we will will produce a colour-magnitude diagrams using the stacked images you made of your open cluster in the previous practical. If you have not yet finished that practical, use the stacked images provided in the ```data``` folder, taken from the open cluster NGC 7789. If you have finished, upload your own stacked images to the ```data``` folder and use them instead.

## Method

In the lecture we saw that, for any star, the difference between the calibrated magnitude, $m$ and the above-atmosphere instrumental magnitude $m_{0,i}$ is given by:
$$m = m_{0,i} + m_{\rm zp},$$
where $m_{\rm zp}$ is known as the zero point. The above-atmosphere instrumental magnitude is given by:
$$m_{0,i} = m_i - kX,$$
where $m_i = -2.5 \log_{10} \left( N_t/t_{\rm exp} \right)$ is the instrumental magnitude, $k$ is the extinction coefficient, and $X$ is the airmass. Therefore:
$$m = m_i - kX + m_{\rm zp}.$$
In other words, if we were to plot the calibrated magnitude $m$ against instrumental magnitude $m_i$ for all the stars in our image, we would expect a straight line, with a gradient of one, and an intercept equal to $ -kX + m_{\rm zp} $. **This intercept could then be added to all of our instrumental magnitudes to produce calibrated magnitudes.**

What makes this technique possible, is the existence of large sky surveys, which have provided calibrated magnitudes for many, relatively bright stars over a very wide areas of the sky. If your data is covered by one of these surveys, and it provides calibrated magnitudes in the same filter as your data, you can apply this technique relatively quickly.

## Accuracy

The accuracy this technique can achieve is limited by two factors. First of all we are not taking into account any secondary effects, such as 2nd-order extinction, or colour terms. Secondly, our photometry cannot be any more accurate than that in the sky survey we use. Typically, large sky surveys achieve accuracies of a few percent. Ignoring the secondary effects will probably introduce errors on a similar level. If you need calibrated photometry to better than a few percent, you will have to observe standard stars, and apply the rigorous method described in the lecture.

## Steps

Applying this method requires the application of four steps. These steps are:

1. finding the stars in our image, and calculating instrumental magnitudes;
1. matching our stars against a sky-survey, so we know instrumental and calibrated magnitude for many stars;
1. calculating the offset between instrumental and calibrated magnitudes, and applying to all stars in our image.


We will carry out these steps in Python. As usual we will draw on several third-party Python libraries. I will explain their use, and provide links to detailed documentation. We will also use some code that I have written which will help ease the code writing.

# Photometry with Photutils

We have done aperture photometry [before](http://slittlefair.staff.shef.ac.uk/teaching/phy241/practicals/P05/index.html), using AstroImageJ. Whilst this tool is excellent for relative photometry, and producing light curves, each aperture must be placed by hand, which makes it onerous to use when we want to measure every star in an image. 

There are a lot of steps to aperture photometry. We must detect stars in the image, and measure their positions accurately. Then we have to add up the counts within a *target aperture*. Finally we have to measure the sky background level in a *sky annulus*, and subtract off the sky background. You can imagine that writing the functions to do this from scratch is going to be hard. Thankfully, there is a Python library called [photutils](http://photutils.readthedocs.org/en/latest/) that is written to do exactly this. 

Photutils is not installed by default on CoCalc. The code cell below will install it. Run this cell, and if there are any **errors**, ask for help (warnings are ok).

In [0]:
import sys
!{sys.executable} -m pip install photutils

In [0]:
## if this code cell runs without error, you have successfully installed photutils!
import photutils as p

<section class="challenge panel panel-success"> 
<div class="panel-heading">
<h2><span class="fa fa-pencil"></span>Read in your image</h2>
</div>
</section>

Using the code cell below, read in your 300s, V-band stacked image into an array called `data`. Read the header from the FITS file into a variable called `header`. It's important to use those variable names - I'll assume they exist later in the notebook.

You can look at the code from Session 5 about the `astropy.io.fits` library if you can't remember how to do this.

In [0]:
# YOUR CODE HERE

<section class="challenge panel panel-success"> 
<div class="panel-heading">
<h2><span class="fa fa-pencil"></span>Creating a Source List</h2>
</div>
</section>

Our first job is to detect our sources. We will do this using an algorithm called DAOFIND ([Stetson 1987, PASP, 99, 191](https://ui.adsabs.harvard.edu/abs/1987PASP...99..191S/abstract)). DAOFIND looks for bright regions in the image that have a peak brightness greater than some threshold and that have a size and shape similar to a Gaussian of specified FWHM.

Stars in our image will stand out above the background, and DAOFIND will find them, but we need to know what threshold to use. One way of doing this is to measure the statistics of the **background** in our image. If we measure the average value of the background, and the amount the background varies, we can look for regions that are significantly brighter than background pixels. 

Below I do that using a "sigma-clipped" mean - where I estimate the average background and the standard deviation. We then throw away all the pixels more than 3 standard deviations (sigma) away from the mean, and repeat the process. I carry on until no pixels are more than 3 standard deviations away from the average value. Then I calculate the mean, median and standard deviation of the remaining pixels. Sound complex? Don't worrry, there's already code to to it!

In [0]:
# import sigma_clipping function from astropy
from astropy.stats import sigma_clipped_stats

mean_background, median_background, background_standard_deviation = sigma_clipped_stats(data, sigma=3.0)

print("The background has an average value of {:.1f} and a standard deviation of {:.1f} counts".format(
    mean_background, background_standard_deviation))

# Finding Stars

Now we know how bright our background is, and how much it varies, let's look for stars that are brighter than the background plus 5 standard deviations. That should be enough that we don't identify bright background pixels as stars by accident. The DAOFIND algorithm needs a guess for how big the stars are - as a Gaussian FWHM - we'll guess at 3 pixels for now.

In [0]:
from photutils import DAOStarFinder

# make a star finder object to look for stars with FWHM~3 pixels that are more than 5-sigma above background
daofind = DAOStarFinder(fwhm=3.0, threshold=5*background_standard_deviation)

# use it to find stars. We'll subtract the background off first, so background pixels have an average value of 0
sources = daofind(data - median_background)

print(sources)

The ``sources`` variable contains a table of all the detected stars. There are various columns, but the ones we are interested in is the X and Y positions of the stars, which you can find with ```sources['xcentroid']``` and ```sources['ycentroid']```.

But how do we know we've found most of the stars? Or if we are mistakenly identifying bright background pixels as stars? We can inspect our sources by-eye. To make this easier, I've written a little helper function to plot your image with the sources overlaid. The function is in the file `photometry_helpers.py`

In [0]:
from photometry_helpers import plot_sources
help(plot_sources)
    
plot_sources(data, sources, 3)

<section class="challenge panel panel-success"> 
<div class="panel-heading">
<h2><span class="fa fa-pencil"></span>Tweak the detection settings</h2>
</div>
</section>

> The DAOFIND algorithm requires a threshold for star detection, and a typical FWHM of the stars in the images. Try different settings for these values, and see how they affect the detection of stars in your data. Make a decision about what values to use for this image.

<section class="challenge panel panel-success"> 
<div class="panel-heading">
<h2><span class="fa fa-pencil"></span>FWHM of stars in the image</h2>
</div>
</section>

We will also need an estimate for the FWHM of stars in the image. We can estimate this by-eye by plotting a bright, isolated, star and plotting the brightness against distance from the star's centre. I've written a clever little routine to do this. 

Using the plot created by the code below, estimate the FWHM of the star in pixels. You should be able to convert this to a value in arcseconds, using the information you recorded during observing, and the specifications of the [Hicks Observatory](https://sites.google.com/sheffield.ac.uk/astronomy/hicks-observatory)

In [0]:
from photometry_helpers import measure_FWHM

measure_FWHM(data, sources)

# Aperture Photometry

So, we have a list of detected sources and positions in the image, and an idea of the FWHM of stars in the image. Now we can perform aperture photometry on all of these sources. I've written a function to do this for us. The comments in the function make it relatively easy to understand, so by all means take a look in the file `photometry_helpers.py` to see what the function does if you like. In brief the function performs the following steps:

1. Add up the counts from each source within a target aperture
2. Measure the sky brightness around each source using the sky annulus
3. Subtract the sky contribution from the counts in step 1.
4. Calculate instrumental magnitude from the counts and exposure time.

By now you should be getting quite good at reading documentation for functions you import, so let's import my function at look at the documentation.

In [0]:
from photometry_helpers import aperture_photometry
help(aperture_photometry)

<section class="challenge panel panel-success"> 
<div class="panel-heading">
<h2><span class="fa fa-pencil"></span>Perform Aperture Photometry</h2>
</div>
</section>

Using the help above, use this function to perform aperture photometry on all of your sources. You'll need to pick values for the aperture radii, remembering everything we've discussed in the course about how these choices affect the quality of your measurement. 

Target apertures want to be big enough to accept a decent fraction of the flux, but not so large that the measurements are very noisy, or contaminated by nearby stars. As a rule of thumb this aperture might have a radius of 1.5-2x the FWHM.

Sky Annuli want to be wide enough to accurately measure the sky, but not so large that the annuli overlap nearby stars.

To get started, try values around 5, 10, 20 pixels for these apertures. We will tweak them later on.

In [0]:
# YOUR CODE HERE. USE THE FUNCTION ABOVE TO PERFORM APERTURE PHOTOMETRY

# SAVE THE TABLE RETURNED IN A SUITABLY NAMED VARIABLE. Perhaps call it V300.

# Photometric Calibration

Now we have a table of instrumental magnitudes (and much besides) in it. The next step is photometric calibration. As a reminder, this involves fitting a straight line to a graph of *instrumental magnitude* against *calibrated magnitude*, to find the offset between the two. Equivalently, we can find the average value of the *difference* between the calibrated and instrumental magnitudes for all our stars.

Since we have instrumental magnitudes and sky positions (RA, Dec) for a number of stars, we must find the matching stars in an online catalog of calibrated magnitudes. We will use the [APASS](https://www.aavso.org/apass) catalog; a catalog which combines several other sky surveys to provide data in many filters across much of the sky. Crucially, in this case it includes B and V magnitudes, the two filters used for our photometry.

To perform the cross-matching we will use the [astroquery](https://astroquery.readthedocs.io/en/latest/) Python library, and we have our aperture photometry results in a [Table](http://docs.astropy.org/en/stable/table/) object from [astropy](http://www.astropy.org). 

In [0]:
from astropy import units as u
from astroquery.xmatch import XMatch

Our photometry is stored as an astropy table, which is a handy object for reading and writing tabular data. These astropy tables play nicely with Jupyter notebooks, so you can simply type the name of the table in a code cell to see the table displayed in the browser. So, if you named your photometry table above ```V300``` the following cell will work. If not, replace the variable name below:

In [0]:
V300

The important columns for our uses are the measured centres of our stars in RA and Dec (**RA** and **DEC**) and the instrumental magnitude and uncertainty (**instrumental_mag** and **e_instrumental_mag**). Note that the magnitude uncertainty is calculated using the CCD signal-to-noise equation we saw in the [lectures](http://slittlefair.staff.shef.ac.uk/teaching/phy241/lectures/L09/index.html).

<section class="challenge panel panel-success"> 
<div class="panel-heading">
<h2><span class="fa fa-pencil"></span>X-match with APASS</h2>
</div>
</section>

> We need to calibrate our photometry, which will involve comparing our instrumental magnitudes to calibrated magnitudes measured for the same stars. We need to match our detected stars with those catalogued in the sky survey [APASS](https://www.aavso.org/apass). We are looking for stars who's RA and Dec matches to within some radius. A service called **Vizier** hosts online versions of astronomical catalogs, and we can use the ```Xmatch.query``` function to match an astropy table with a table hosted by Vizier using the code below.

> Run the code cell below, and note carefully how we specify the columns that contain RA and Dec in our *local* table, and how we set the maximum distance for a valid match. **II/336/apass9** is the name of the APASS catalog on Vizier. If you need to find the names of other catalogs (perhaps APASS doesn't cover the patch of sky containing your open cluster), you can enter the catalog name in the search box [here](http://vizier.u-strasbg.fr)

> **The code cell below may take a while to run. Be patient...**

In [0]:
xmatch = XMatch.query(cat1=V300, cat2='vizier:II/336/apass9', max_distance=2*u.arcsec, colRA1='RA', colDec1='DEC')

The result of this query (```xmatch```) is also an astropy ```Table```. It has all the columns from both tables for each valid match. If you want to extract a column and save it into a variable, you can access the ```Table``` like a dictionary. So the code below extracts the **Magnitude** column (which is instrumental magnitude from our APT photometry) and **Vmag** column from UCAC4 and computes the difference.

In [0]:
delta_mag = xmatch['Vmag'] - xmatch['instrumental_mag']

<section class="challenge panel panel-success"> 
<div class="panel-heading">
<h2><span class="fa fa-pencil"></span>Plot the difference and find the zero-point</h2>
</div>
</section>

> Plot the difference between instrumental and calibrated magnitude found above, against the calibrated V-band magnitude on the X-axis. You should see something like the figure below:

<img src="../../images/V_zeropoint.png" style="margin: 0px" width=750px/>

In [0]:
# YOUR CODE HERE - MAKE A PLOT LIKE THE ONE ABOVE. DON'T WORRY ABOUT ADDING THE HORIZONTAL LINE

The magnitude difference between instrumental and calibrated magnitude *should* be a constant, which is the value of $kX + m_{\rm zp}$. In the figure above, you can see there are plenty of outlying points, and the bright stars deviate from the constant. The outliers are either stars whose photometry is bad (poor sky estimation, or contaminated by very close stars), or spurious detections (i.e not stars). The deviation of the bright stars is caused because they are [saturated](http://slittlefair.staff.shef.ac.uk/teaching/phy241/lectures/L06/index.html#readout), and so we cannot accurately measure their flux.

Since $m = m_i - kX + m_{\rm zp}$, we can find the value of $-kX + m_{\rm zp}$. - which I'll call the *zeropoint* from now on - by calculating the **median** difference between the instrumental and calibrated magnitude. The median will be robust against the outliers - but we only want to do it for the non-saturated stars!

<section class="challenge panel panel-success"> 
<div class="panel-heading">
<h2><span class="fa fa-pencil"></span>Calculate the zero-point</h2>
</div>
</section>

> Calculate the median value of ```delta_mag```. **If you have evidence for saturated stars**, use a NumPy *mask* to only calculate the median of stars that are not saturated (see notes on Fancy slicing in the NumPy notebook).

> The median value of ```delta_mag``` is our estimate of $-kX + m_{\rm zp}$ - i.e the value we want to add to our instrumental magnitudes to get a calibrated magnitude. You can add this value to the **instrumental_mag** column of the V300 table easily using ```V300['calibrated_mag'] = V300['instrumental_mag'] + zp```, where ```zp``` is the median value you found.

In [0]:
# YOUR CODE HERE. CALCULATE THE ZEROPOINT AND ADD IT TO THE MAGNITUDE COLUMN OF THE V300 table

---------
<section class="challenge panel panel-success"> 
<div class="panel-heading">
<h1>Homework #6</h1>
<h2><span class="fa fa-pencil"></span>The B-band data and making a CMD</h2>
</div>
</section>

> Your task in the homework is to repeat the steps above for the rest of your data, and make two CMDs, one for the 30s data, and one for the 300s data.

<section class="challenge panel panel-success"> 
<div class="panel-heading">
<h2><span class="fa fa-question"></span>  The B-band data (4 points)</h2>
</div>
</section>

> Repeat the steps above for the 300-second B-band data. Make a plot of the difference between instrumental B-band magnitude and APASS B-band magnitude. Calculate the offset needed to correct your instrumental mags (zeropoint) and display it on your plot.

> Finally, add your zeropoint to the **instrumental_mag** column of the 300s B-band table to make a new **calibrated_mag** column.

In [0]:
# YOUR CODE HERE

Now you should have a two astropy tables in computer memory. One with corrected (calibrated) V-band magnitudes and one with corrected (calibrated) B-band magnitudes. We need to cross-match these tables with each other, to find which stars in the V-band table match with stars in the B-band table. We can't use ```astroquery```'s Xmatch for this, since they are both local tables. Instead, we will use the ```SkyCoord``` object from astropy, which is meant to work with coordinates on the sky. You can create a ```SkyCoord``` object with positions of all of the stars in a table like so:

```python
from astropy.coordinates import SkyCoord

coo_v = SkyCoord(V300['RA'], V300['DEC'], unit=u.deg)
```

The ```SkyCoord``` you made is a bit like a NumPy array, but it is an array of positions on the sky. It carries within it several useful functions, including one to match positions against another sky coordinate object - ```match_coordinates_sky```. It's usage is shown below, and the documentation is [here](http://docs.astropy.org/en/stable/coordinates/matchsep.html#matching-catalogs). ```match_coordinates_sky``` returns an array of indices and the 2D and 3D separations of the matches. The array of indices can be used as a slice to sort the second set of coordinates, or the table from which it came, so that each row of the sorted coordinate/table contains the closest match of the corresponding row. Perhaps an example will explain:

```python
# match every entry in coo_v with the nearest entry in coo_v
idx, distance_2d, distance_3d = coo_v.match_to_catalog_sky(coo_b)

# using idx as a slice for the table B300 will sort it so that B300[0] is the closest match to V300[0]
B300 = B300[idx]
```

We are not quite done yet though - we have found the closest match to every object in the V300 table, but some of those matches may be quite far away. We could create a *fancy slicing mask* of True and False values by comparing the ```distance_2d``` array to some threshold separation, and use that mask to remove the rows where there is no close cross-match:

```python
mask = distance_2d < 3 * u.arcsec
B300 = B300[mask]
V300 = V300[mask]
```

<section class="challenge panel panel-success"> 
<div class="panel-heading">
<h2><span class="fa fa-question"></span>Cross-matching your V and B-band tables (2 points)<h2>
</div>
</section>

> Create ```SkyCoord``` objects from your 300s B and V tables. Match them using ```match_coordinates_sky```, and use a mask to remove all rows where there is no good match.

In [0]:
# YOUR CODE HERE

<section class="challenge panel panel-success"> 
<div class="panel-heading">
<h2><span class="fa fa-question"></span>Plotting your CMD (3 points)</h2>
</div>
</section>

> With your matched tables in hand, extract the calibrated magnitude columns from each into two arrays called ```V``` and ```B```. Calculate ```B-V``` and plot a colour-magnitude diagram of ```B-V``` against ```V```. Your plot should look something like the one below, which is for the cluster NGC 7789.

<img src="../../images/CMD.png" style="margin: 0px" width=750px/>

> Also calculate error bars on ```B-V```: remember that $\Delta (x+y)^2 = \Delta x^2 + \Delta y^2$

> Hint: you can reverse the y-axis with ```plt.gca().invert_yaxis()```

In [0]:
# YOUR CODE HERE

<section class="challenge panel panel-success"> 
<div class="panel-heading">
<h2><span class="fa fa-question"></span>Save your data (1 point)</h2>
</div>
</section>

> Save your ```V``` and ```B-V``` data, along with their uncertainties, to a text file, for use in the next lab!

> You might want to revisit Session1 for instructions for writing data to a file. Or, you could create an astropy Table using the instructions [here](https://docs.astropy.org/en/stable/table/construct_table.html) and [here](https://docs.astropy.org/en/stable/table/io.html)


In [0]:
# YOUR CODE HERE